In [1]:
import sys
sys.path.append('/home/zhw272/programming/pipeline_utility/in_development/Will')
from cell_extractor.CellDetectorBase import CellDetectorBase
from abakit.lib.annotation_layer import Cell,random_string
import numpy as np
from random import sample
from abakit.lib.UrlGenerator import UrlGenerator

def numpy_to_json(data,color_hex = None):
    cells = []
    for coord in data:
        cell = Cell(np.array(coord,dtype=float),random_string())
        cell.category = 'Round3'
        # del cell.description
        cell_json = cell.to_json()
        if color_hex is not None:
            cell_json["props"] = [color_hex]
        cells.append(cell_json)
    return cells

In [3]:
detections.head()

,animal,section,row,col,label,mean_score,std_score,predictions
0,DK41,347,5364,28747,0.0,-3.753175,1.660697,-2
1,DK41,347,5324,28755,0.0,-4.274146,1.619854,-2
2,DK41,347,5334,28704,0.0,-8.031691,1.333832,-2
3,DK41,347,5337,28674,0.0,-4.631068,0.967737,-2
4,DK41,347,5338,28678,0.0,-8.720782,1.677284,-2


In [2]:
animal = 'DK41'
base = CellDetectorBase(animal,round = 2)
detections = base.load_detections()
sure = detections[detections.predictions==2]
unsure = detections[detections.predictions==0]
len(sure),len(unsure),len(detections)

(5649, 4491, 2604854)

In [3]:
sure_data = sure[['col','row','section']].sample(500).sort_values('section').to_numpy().tolist()
unsure_data = unsure[['col','row','section']].sample(831).sort_values('section').to_numpy().tolist()
sure_cells = numpy_to_json(sure_data)
unsure_cells = numpy_to_json(unsure_data,color_hex='#1d66db')
urlGen = UrlGenerator()
image_layer = 'precomputed://https://activebrainatlas.ucsd.edu/data/'+animal+'/neuroglancer_data/'
urlGen.add_stack_image(animal,channel=1)
urlGen.add_stack_image(animal,channel=2,color='red')
urlGen.add_stack_image(animal,channel=3,color='green')
urlGen.add_annotation_layer('Sure',annotations = sure_cells)
urlGen.add_annotation_layer('Unsure',annotations = unsure_cells)
urlGen.add_to_database('test QC3',34)

573

In [4]:
from abakit.lib.Controllers.SqlController import SqlController

controller = SqlController('DK52')
controller.get_resolution('DK52')*(10^6)

array([  3.9,   3.9, 240. ])

In [5]:
controller.get_resolution('DK52')*10^6

TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''